In [0]:
%sql
create schema if not exists inflacao.gold;

In [0]:
%sql

CREATE OR REPLACE TABLE inflacao.gold.gold_comparativo_investimentos AS
WITH 
parametros AS (
    SELECT 
        10000.00 AS investimento_inicial,
        date_add(current_date(), -1825) AS data_inicio_analise
),

assets_calc AS (
    SELECT 
        a.data_referencia,
        a.ativo,
        a.preco_brl,
        FIRST_VALUE(a.preco_brl) OVER (PARTITION BY a.ativo ORDER BY a.data_referencia ASC) AS preco_inicial
    FROM inflacao.silver.silver_assets_brl a
    CROSS JOIN parametros p
    WHERE a.data_referencia >= p.data_inicio_analise 
),
assets_final AS (
    SELECT
        a.data_referencia,
        a.ativo,
        (a.preco_brl / a.preco_inicial) * c.investimento_inicial AS valor_patrimonio
    FROM assets_calc a
    CROSS JOIN parametros c
),

poupanca_calc AS (
    SELECT
        s.data_referencia,
        'Poupanca' AS ativo,
        s.taxa_mensal_decimal,
        SUM(LN(1 + s.taxa_mensal_decimal)) OVER (ORDER BY s.data_referencia ASC) AS soma_log_acumulada
    FROM inflacao.silver.silver_poupanca_clean s
    CROSS JOIN parametros p
    WHERE s.data_referencia >= p.data_inicio_analise 
),
poupanca_final AS (
    SELECT
        p.data_referencia,
        p.ativo,
        EXP(p.soma_log_acumulada) * c.investimento_inicial AS valor_patrimonio
    FROM poupanca_calc p
    CROSS JOIN parametros c
)

SELECT * FROM assets_final
UNION ALL
SELECT * FROM poupanca_final
ORDER BY data_referencia DESC, ativo;